In [7]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import dataloader

from preprocessing import Normalize_df, WindSpeedTimeSeriesDataset, ComposeTransform, ToTensor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
dataset = Normalize_df(pd.read_csv('./dataset-daily.csv'))
dataset.head()

,time,air_temperature_mean,pressure,wind_direction,wind_speed
0,0.000000,0.370203,0.103164,0.732591,0.625000
1,0.000011,0.322799,0.268912,0.838440,0.354167
2,0.000022,0.302483,0.709078,0.988858,0.260417
3,0.000033,0.246050,0.850758,0.239554,0.093750
4,0.000044,0.194131,0.827372,0.345404,0.291667


In [14]:
from_v = 0
to_v = 200

#plt.plot(dataset['air_temperature_mean'].values[from_v:to_v], label = 'air_temperature_mean')
#plt.plot(dataset['pressure'].values[from_v:to_v], label = 'pressure')
plt.plot(dataset['wind_direction'].values[from_v:to_v], label = 'wind_direction')
plt.plot(dataset['wind_speed'].values[from_v:to_v], label = 'wind_speed')
plt.legend()
plt.show()

In [4]:
train_dataset[0]

(tensor([[0.3702, 0.1032, 0.7326],
         [0.3228, 0.2689, 0.8384],
         [0.3025, 0.7091, 0.9889],
         [0.2460, 0.8508, 0.2396],
         [0.1941, 0.8274, 0.3454],
         [0.1670, 0.6671, 0.3398]], dtype=torch.float64),
 tensor([[0.6250, 0.3542, 0.2604, 0.0938, 0.2917, 0.2396]], dtype=torch.float64))

In [3]:
train_dataset = WindSpeedTimeSeriesDataset(dataset, window_size=6, transform=ComposeTransform([ToTensor()]))

In [5]:
batch_size = 32

trainloader = dataloader.DataLoader(train_dataset, batch_size, shuffle = True)

In [6]:
f, l = next(iter(trainloader))
f = f.type(torch.FloatTensor)

print(f'Features Size: {f.shape}')
print(f'Labels Size: {l.shape}')

Features Size: torch.Size([32, 6, 3])
Labels Size: torch.Size([32, 1, 6])


In [7]:
from torch import nn

In [69]:
input_size = 3

conv1 = nn.Conv1d(6, 18, 1)
pool1 = nn.MaxPool1d(1)
        
fc2 = nn.Linear(18 * input_size, 18)
        
lstm3 = nn.LSTM(input_size= 18,
                            hidden_size=36,
                            num_layers=1,
                            batch_first= True,
                            dropout = 0)
fc4 = nn.Linear(36, 18)
fc5 = nn.Linear(18, 6)

In [72]:
hidden = (torch.zeros(1, 32, 36),
                torch.zeros(1, 32, 36))

In [74]:
x = pool1(conv1(f))
        
x = x.view(x.size(0), -1)
x = fc2(x)
x = x.unsqueeze(1)
print(x.shape)
x, hidden = lstm3(x, hidden)
print(x.shape)
x = fc4(x)
x = fc5(x)
print(x.shape)

torch.Size([32, 1, 18])
torch.Size([32, 1, 36])
torch.Size([32, 1, 6])


In [50]:
x = conv1(f)
x.shape

torch.Size([32, 18, 3])

In [51]:
x, hidden = lstm1(x, hidden)
x.shape

torch.Size([32, 18, 30])

In [11]:
x = x.view(x.size(0), -1)
x.shape

torch.Size([32, 54])

In [12]:
x = fc(x)
x.shape

torch.Size([32, 6])

In [13]:
x = x.unsqueeze(1)
x.shape

torch.Size([32, 1, 6])

In [23]:
x, hidden = lstm1(x, hidden)

In [24]:
x.shape

torch.Size([32, 1, 30])

In [26]:
x = fc2(x)

In [28]:
x.shape

torch.Size([32, 1, 50])

In [29]:
fc3 = nn.Linear(50, 6)

In [30]:
x = fc3(x)

In [31]:
x.shape

torch.Size([32, 1, 6])

In [34]:
l = nn.MSELoss()(x, l.type(torch.FloatTensor))

In [35]:
l.backward()

tensor(0.0765, grad_fn=<MseLossBackward>)

tensor(0.0765, grad_fn=<MseLossBackward>)